In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


## Cargo el dataset con las modificaciones iniciales

In [2]:
df = pd.read_csv('./data/data_preprocessing.csv', index_col=0)
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,...,5,0,0,0,0,0,0,3,11,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   int64 

## Primeras modificaciones

In [4]:
# Cambio el índice
df.set_index('ID', inplace=True)

# Cambio tipo a datetime
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'])

# Eliminar columnas
df.drop(['Z_CostContact', 'Z_Revenue'], axis=1, inplace=True)

# Cambiar tipo a categóricas
cols_to_category = ['Education', 'Marital_Status','Complain']
df[cols_to_category] = df[cols_to_category].astype('category')

# Eliminación de YOLO y cambio de nombre a Absurd
df['Marital_Status'] = df['Marital_Status'].cat.rename_categories({'Absurd': 'Others'})
df = df[df['Marital_Status'] != 'YOLO']


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2238 entries, 5524 to 9405
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Year_Birth           2238 non-null   int64         
 1   Education            2238 non-null   category      
 2   Marital_Status       2238 non-null   category      
 3   Income               2214 non-null   float64       
 4   Kidhome              2238 non-null   int64         
 5   Teenhome             2238 non-null   int64         
 6   Dt_Customer          2238 non-null   datetime64[ns]
 7   Recency              2238 non-null   int64         
 8   MntWines             2238 non-null   int64         
 9   MntFruits            2238 non-null   int64         
 10  MntMeatProducts      2238 non-null   int64         
 11  MntFishProducts      2238 non-null   int64         
 12  MntSweetProducts     2238 non-null   int64         
 13  MntGoldProds         2238 non-null 

In [6]:
df.loc[(df['Marital_Status'] == 'Absurd') | (df['Marital_Status']== 'YOLO' )]

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
ID,,,,,,,,,,,,,,,,,,,,,


## Variable target

In [7]:
df['Response'].value_counts(True)

Response
0    0.851206
1    0.148794
Name: proportion, dtype: float64

Se encuentra muy desbalanceada.

## División en train y test

In [8]:
train_set, test_set = train_test_split(df,test_size=0.20,random_state=42,stratify=df['Response'])
print('Train_set',train_set.shape)
print('Test_set', test_set.shape)

Train_set (1790, 26)
Test_set (448, 26)


## Imputación de nulos

Flag con los nulos

In [9]:
train_set['income_missing'] = np.where(train_set['Income'].isnull(),1,0)

Otras variables con correlación

In [10]:
corr_matrix = train_set.corr(numeric_only=True)
abs(corr_matrix['Income']).sort_values(ascending=False) > 0.5
corr_matrix[(corr_matrix['Income'] > 0.5)].index

Index(['Income', 'MntWines', 'MntMeatProducts', 'NumCatalogPurchases',
       'NumStorePurchases'],
      dtype='object')

IterativeImputer para realizar la imputación de nulos

In [11]:
features_iterative = ['Income', 'MntWines', 'MntMeatProducts', 'NumCatalogPurchases']
imputer_iterativo = IterativeImputer()

imputer_iterativo.fit(train_set[features_iterative])

train_set['Income'] = imputer_iterativo.transform(train_set[features_iterative])

# Aplicar al test_set
test_set['Income'] = imputer_iterativo.transform(test_set[features_iterative])

Comprobación

In [12]:
train_set['Income'].isna().sum()

0

## Nuevas variables

Imputación a train_set

In [13]:
# Edad tomando como referencia 2000
train_set['age']= 2015 - train_set['Year_Birth']

# Antigüedad como cliente tomando como referencia 2020
train_set['customes_seniority'] = 2015 - train_set['Dt_Customer'].dt.year

# Miembros totales en casa
dicc_status = {
    'Together': 2,  # Generalmente representa a dos personas viviendo juntas
    'Divorced': 1,  # Generalmente representa a una persona que está divorciada
    'Married': 2,   # Generalmente representa a dos personas casadas
    'Single': 1,    # Generalmente representa a una persona soltera
    'Widow': 1,     # Generalmente representa a una persona viuda
    'Alone': 1,     # Generalmente representa a una persona viviendo sola
    'Others': 1     # Puede variar, pero generalmente representa a una persona
}
train_set['Status_members'] = train_set['Marital_Status'].map(dicc_status)
train_set['Household_members'] = train_set['Status_members']+train_set['Kidhome']+train_set['Teenhome']

# Gasto total
train_set['Total_amount'] = train_set.filter(like='Mnt').sum(axis = 1)

# Nº compras totales
train_set['Total_purchase'] = train_set.filter(like='Purchases').sum(axis = 1)

# Gasto promedio
train_set['Median_amount_purchase'] = train_set['Total_amount']/train_set['Total_purchase']

# Nº ofertas aceptadas
train_set['Total_cmp'] = train_set.filter(like='Accepted').apply(pd.to_numeric, errors='coerce').sum(axis=1)

# % de ofertas aceptadas
train_set_numeric = train_set.filter(like='Cmp').apply(pd.to_numeric, errors='coerce')
train_set['Total_%_cmp'] = train_set['Total_cmp'] / len(train_set_numeric.columns)

# Elimino columnas intermedias
train_set = train_set.drop(columns=['Status_members'])

In [14]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1790 entries, 5675 to 6634
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Year_Birth              1790 non-null   int64         
 1   Education               1790 non-null   category      
 2   Marital_Status          1790 non-null   category      
 3   Income                  1790 non-null   float64       
 4   Kidhome                 1790 non-null   int64         
 5   Teenhome                1790 non-null   int64         
 6   Dt_Customer             1790 non-null   datetime64[ns]
 7   Recency                 1790 non-null   int64         
 8   MntWines                1790 non-null   int64         
 9   MntFruits               1790 non-null   int64         
 10  MntMeatProducts         1790 non-null   int64         
 11  MntFishProducts         1790 non-null   int64         
 12  MntSweetProducts        1790 non-null   int64     

In [15]:
train_set.iloc[:,25:36]

,Response,income_missing,age,customes_seniority,Household_members,Total_amount,Total_purchase,Median_amount_purchase,Total_cmp,Total_%_cmp
ID,,,,,,,,,,
5675,0,0,55,3,2.0,493,22,22.409091,1,0.2
5543,0,0,49,2,3.0,802,25,32.080000,1,0.2
3011,0,0,50,1,3.0,227,11,20.636364,0,0.0
535,0,0,28,1,1.0,778,27,28.814815,0,0.0
10755,0,0,39,2,3.0,81,9,9.000000,0,0.0
...,...,...,...,...,...,...,...,...,...,...
5320,1,0,42,2,3.0,171,12,14.250000,0,0.0
2894,1,0,30,2,1.0,2013,21,95.857143,3,0.6
1726,1,0,45,2,1.0,81,5,16.200000,1,0.2


Imputación a test_set

In [16]:
# Edad tomando como referencia 2000
test_set['age'] = 2015 - test_set['Year_Birth']

# Antigüedad como cliente tomando como referencia 2020
test_set['customes_seniority'] = 2015 - test_set['Dt_Customer'].dt.year

# Miembros totales en casa
dicc_status = {
    'Together': 2,  # Generalmente representa a dos personas viviendo juntas
    'Divorced': 1,  # Generalmente representa a una persona que está divorciada
    'Married': 2,   # Generalmente representa a dos personas casadas
    'Single': 1,    # Generalmente representa a una persona soltera
    'Widow': 1,     # Generalmente representa a una persona viuda
    'Alone': 1,     # Generalmente representa a una persona viviendo sola
    'Others': 1     # Puede variar, pero generalmente representa a una persona
}
test_set['Status_members'] = test_set['Marital_Status'].map(dicc_status)
test_set['Household_members'] = test_set['Status_members'] + test_set['Kidhome'] + test_set['Teenhome']

# Gasto total
test_set['Total_amount'] = test_set.filter(like='Mnt').sum(axis=1)

# Nº compras totales
test_set['Total_purchase'] = test_set.filter(like='Purchases').sum(axis=1)

# Gasto promedio
test_set['Median_amount_purchase'] = test_set['Total_amount'] / test_set['Total_purchase']

# Nº ofertas aceptadas
test_set['Total_cmp'] = test_set.filter(like='Accepted').apply(pd.to_numeric, errors='coerce').sum(axis=1)

# % de ofertas aceptadas
test_set_numeric = test_set.filter(like='Cmp').apply(pd.to_numeric, errors='coerce')
test_set['Total_%_cmp'] = test_set['Total_cmp'] / len(test_set_numeric.columns)

# Elimino columnas intermedias
test_set = test_set.drop(columns=['Status_members'])

In [17]:
train_set.to_csv('./data/train_set.csv')
test_set.to_csv('./data/test_set.csv')